L'architecture Transformer, introduite dans l'article "Attention is All You Need" de Vaswani et al., est une puissante architecture d'apprentissage profond principalement utilisée pour les tâches de traitement du langage naturel (NLP). Elle repose fortement sur le mécanisme de l'attention pour traiter les séquences d'entrée en parallèle.

L'architecture Transformer se compose de deux composants principaux : l'encodeur et le décodeur.

1. **Encodeur :**
   - L'encodeur est responsable du traitement de la séquence d'entrée et de sa conversion en une série de représentations de caractéristiques qui capturent l'information dans la séquence.
   - Il se compose d'une pile de couches identiques. Chaque couche a deux sous-couches :
      - **Auto-attention multi-têtes :** Cette sous-couche permet à l'encodeur d'évaluer l'importance des différents mots dans la séquence d'entrée les uns par rapport aux autres. Elle calcule les scores d'attention entre chaque paire de mots dans la séquence d'entrée, aboutissant à une combinaison pondérée de tous les mots.
      - **Réseaux d'Avancement à Travers les Positions :** Cette sous-couche applique un réseau neuronal à propagation avant à chaque position de manière séparée et identique.
   - Les sorties de ces sous-couches passent par une connexion résiduelle suivie d'une normalisation de couche.



In [ ]:
def clone_layers(module, num):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(num)])

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, vocab_size, d_model, num, n_heads):
        super(EncoderBlock, self).__init__()
        self.num = num
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pe = Pos_Encoder(d_model, max_len= 80)
        self.layers = clone_layers(EncoderLayer(d_model, n_heads), num)
        self.norm = LayerNorm(d_model)

    def forward(self, src, mask):
        x = self.embed(src);
        x = self.pe(x)
        for i in range(self.num):
            x = self.layers[i](x, mask)
        x = self.norm(x)
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, droprate=0.1):
        super(EncoderLayer, self).__init__()
        self.norm1 = LayerNorm(d_model)
        self.attn = MHAttention(d_model=d_model, n_heads=n_heads)
        self.dropout1 = nn.Dropout(droprate)
        self.norm2 = LayerNorm(d_model)
        self.fc1 = FeedForward(d_model=d_model)
        self.dropout2 = nn.Dropout(droprate)

    def forward(self, x, mask):
        x1 = self.norm1(x);
        x = x + self.dropout1(self.attn(x1, x1, x1, mask));
        x1 = self.norm2(x)
        x = x + self.dropout2(self.fc1(x1));
        return x

2. **Décodeur :**
   - Le décodeur est responsable de la génération de la séquence de sortie en se basant sur l'information traitée provenant de l'encodeur.
   - Il se compose également d'une pile de couches identiques, mais avec une sous-couche supplémentaire :
      - **Auto-attention multi-têtes :** Similaire à l'encodeur, cette sous-couche permet au décodeur de se concentrer sur différentes parties de la séquence d'entrée. Cependant, elle dispose d'un mécanisme de masquage pour empêcher le décodeur de regarder les positions futures dans la séquence de sortie.
      - **Attention de l'encodeur au décodeur multi-têtes :** Cette sous-couche aide le décodeur à se concentrer sur différentes parties de la séquence d'entrée encodée. Elle permet au décodeur de s'attarder sur différentes positions dans la séquence d'entrée en fonction de l'importance de chaque position pour générer le prochain mot.
      - **Réseaux d'Avancement à Travers les Positions :** Identique à celui de l'encodeur.
   - Tout comme pour l'encodeur, les sorties de ces sous-couches passent par une connexion résiduelle suivie d'une normalisation de couche.

3. **Encodage de Position :**
   - Étant donné que l'architecture Transformer n'a pas de compréhension inhérente de l'ordre des éléments dans une séquence, des encodages de position sont ajoutés aux embeddings d'entrée pour donner au modèle des informations sur les positions relatives ou absolues des jetons dans la séquence.

4. **Couches Linéaires et Softmax Finales :**
   - La sortie de la dernière couche de décodeur passe par une couche linéaire pour obtenir les logits (scores) pour chaque jeton possible dans le vocabulaire de sortie.
   - Une fonction softmax est ensuite appliquée à ces logits pour obtenir une distribution de probabilité sur le vocabulaire, indiquant la probabilité de chaque jeton d'être le mot suivant dans la séquence.

Pendant l'entraînement, le modèle est optimisé pour minimiser la différence entre les probabilités prédites et les séquences cibles réelles. Cela se fait généralement en utilisant une fonction de perte telle que la perte de cross-entropie.

Pendant l'inférence (ou la génération), le modèle est utilisé de manière auto-régressive, ce qui signifie qu'il génère un jeton à la fois en utilisant ses propres prédictions comme entrée pour l'étape suivante, jusqu'à ce qu'une condition d'arrêt soit atteinte (par exemple, une longueur maximale ou un jeton de fin de séquence). En somme, l'architecture Transformer exploite les mécanismes d'auto-attention et les réseaux neuronaux à propagation avant pour traiter les séquences d'entrée en parallèle, ce qui la rend très efficace pour un large éventail de tâches de NLP.

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, droprate=0.1):
        super(DecoderLayer, self).__init__()
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.norm3 = LayerNorm(d_model)
        self.dropout1 = nn.Dropout(droprate)
        self.dropout2 = nn.Dropout(droprate)
        self.dropout3 = nn.Dropout(droprate)
        self.attn1 = MHAttention(d_model=d_model, n_heads=n_heads)
        self.attn2 = MHAttention(d_model=d_model, n_heads=n_heads)
        self.fc1 = FeedForward(d_model=d_model)

    def forward(self, x, e_out, src_mask, trg_mask):
        x1 = self.norm1(x);
        x = x + self.dropout1(self.attn1(x1, x1, x1, trg_mask));
        x1 = self.norm2(x)
        x = x + self.dropout2(self.attn2(x1, e_out, e_out, src_mask));
        x1 = self.norm3(x)
        x = x + self.dropout3(self.fc1(x1));
        return x

class DecoderBlock(nn.Module):
    def __init__(self, vocab_size, d_model, num, n_heads):
        super(DecoderBlock, self).__init__()
        self.num = num
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pe = Pos_Encoder(d_model, max_len = 80)
        self.layers = clone_layers(DecoderLayer(d_model, n_heads), num)
        self.norm = LayerNorm(d_model)

    def forward(self, trg, e_out, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.num):
            x = self.layers[i](x, e_out, src_mask, trg_mask)
        x = self.norm(x)
        return x


# Transformer

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, num, n_heads):
        super(Transformer, self).__init__()
        self.encoder = EncoderBlock(vocab_size=src_vocab, d_model=d_model, num=num, n_heads=n_heads)
        self.decoder = DecoderBlock(vocab_size=trg_vocab, d_model=d_model, num=num, n_heads=n_heads)
        self.fc1 = nn.Linear(d_model, trg_vocab)

    def forward(self, src, trg, src_mask, trg_mask):
        e_out = self.encoder(src, src_mask);
        d_out = self.decoder(trg, e_out, src_mask, trg_mask);
        x = self.fc1(d_out);
        return x